In [134]:
###10분짜리 6초윈도우 슬라이드하면서
import pickle
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import sklearn
from sklearn import svm
from datetime import datetime
import glob

filename_queue=glob.glob('./rawdata/compdata[1-3].csv')
#feature extration from data
def feature_extraction(window):
    mean=window.mean(axis=-1)
    stddev=window.std(axis=-1)
    median=np.median(window,axis=-1)
    percent25=np.percentile(window,25,axis=-1)
    percent75=np.percentile(window,75,axis=-1)
    return(np.array([mean,stddev,median,percent25,percent75]))

#data[0:600] 각각 1sec
#windowSize=6sec=6
#stepSize=1sec=1
#len(data)=600=600sec=10minute
def sliding_window(data,stepSize,windowSize):
    for x in range(0,len(data)-windowSize+1,stepSize):
        yield(x,data[x:x+windowSize])

def pred_window(clf,data):
    test_set=[]
    for(x,window) in sliding_window(data,1,6): 
        wind=window.reshape(180)
        test_set.append(wind)
    print(np.shape(test_set))
    return clf.predict(test_set)

In [123]:
#showing shape of the data
for file in filename_queue:
    df=pd.read_csv(file)
    print(file,df.shape)



./rawdata/compdata2.csv (21, 4503)
./rawdata/compdata1.csv (20, 4503)
./rawdata/compdata3.csv (43, 4503)


In [131]:
return_path="./featuredata/compfeature.csv"
file=open(return_path,'w')
writer=csv.writer(file)
#writer.writerows([["context"],["data"]])  
#showing shape of the data
for filename in filename_queue:
    file=open(filename, newline='')
    reader=csv.reader(file)
    index=0
    #print(df.head())
    i=0
  
    ndata=[]
    X_acc=[]
    Y_acc=[]
    Z_acc=[]
    X_gyro=[]
    Y_gyro=[]
    Z_gyro=[]
    
    ##writer.writerows([X_acc,Y_acc,Z_acc,X_gyro,Y_gyro,Z_gyro])


    for row in reader:
        row[3:]=[float(i) for i in row[3:]]
        i+=1
        if(i>20):
            break
        if(int(row[2])==0):
            time=int(row[0]) #minutewise
            context=[row[1]] #drink 0, eat 1, cafe 2, desk 3
            sensor=int(row[2]) #acc:0 gyro:2
            X_acc=row[3:1503] #1500 =60sec/40ms
            Y_acc=row[1503:3003]
            Z_acc=row[3003:4503]
        if(int(row[2])==1):
            X_gyro=row[3:1503]
            Y_gyro=row[1503:3003]
            Z_gyro=row[3003:4503]
            list=[X_acc,Y_acc,Z_acc,X_gyro,Y_gyro,Z_gyro]
            for x in range(0,60):
                wind=np.array([X_acc[25*x:25*x+25],Y_acc[25*x:25*x+25],Z_acc[25*x:25*x+25],
                            X_gyro[25*x:25*x+25],Y_gyro[25*x:25*x+25],Z_gyro[25*x:25*x+25]])  
                feature=feature_extraction(wind)
                feature=feature.reshape(6,5)
                ndata.append(feature)
    ndata=np.array(ndata)  #600, 6, 5 # format to be predicted
    flatten_data=data.flatten() #18000, format to be saved
    row_data=flatten_data
    writer.writerows([context,row_data])



In [133]:
# load svm
filename='./svm.sav'
loaded_model = pickle.load(open(filename, 'rb'))
print(pred_window(loaded_model))

(595, 180)
[ 0 14 14 14 14 14 14 14 14  0  0  0  0  0  0 14 14 14 14 14 14 14 14  3
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14  0  0  0  0  0  0  0 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14  0  0  0  0  0  0  0  0  0  0  0 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14  0 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14  5  5  0  5 14  3 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14  0  0  0  0  0  0  0
  0  0  0  0  0  0 14 14 14 14 14 14 14 14  1  0  5  5  0 14 14  1 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14  0 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14 14
 14 14 14 14 14 14 14 14 14 14 14 14 14 